In [1]:
import os
import torch
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import torchvision
from torch import nn,optim
from torch.nn import functional as F
from torchvision import transforms 
from torchvision import models as m
from torch.utils.data import DataLoader
import torch
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import time
import datetime
import gc  #garbage collector
import json
from timm.models.vision_transformer import VisionTransformer
from torchinfo import summary
import cv2

In [3]:
torch.cuda.manual_seed(1412)
torch.cuda.manual_seed_all(1412)
torch.manual_seed(420)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

True


In [5]:
vgg16_bn_ =m.vgg16_bn() 
resnet18_ = m.resnet18()

In [6]:
class MyNet3(nn.Module):
    def __init__(self,in_features=3):
        super().__init__()
#         self.conv1=nn.Sequential( BasicConv2d(in_ = in_features,out_=32,kernel_size=3,stride=1,padding=1)
#                                  ,BasicConv2d(in_ = 32,out_=64,kernel_size=3,stride=1,padding=1)
#                                 ,nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False))
#         self.conv1=nn.Sequential( BasicConv2d(in_ = in_features,out_=32,kernel_size=3,stride=1,padding=1)
#                                  ,BasicConv2d(in_ = 32,out_=64,kernel_size=3,stride=1,padding=1)
#                                 ,nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False))
    
#         self.conv1 = nn.Sequential(BasicConv2d_two(in_features,32,64,kernel_size=3,stride=1,padding=1))
        self.conv1 = nn.Sequential(BasicConv2d_two( in_features, 32, 32,kernel_size=3,stride=1,padding=1))
        self.conv2 = nn.Sequential(BasicConv2d_two( 32, 64, 64,kernel_size=3,stride=1,padding=1))
#         self.block1=vgg16_bn_.features[7:14]
        self.block1=resnet18_.layer2
        self.block2=resnet18_.layer3
        self.avgpool=vgg16_bn_.avgpool
        self.classifer=nn.Sequential( nn.Linear(12544, 256, bias=True)
                                     ,nn.BatchNorm1d(256)
                                     ,nn.Dropout(p=0.5)
                                     ,nn.Linear(256,128, bias=True)
                                     ,nn.BatchNorm1d(128)
                                     ,nn.Dropout(p=0.5)
                                    )
        self.fc = nn.Linear(128,5)
        
    def forward(self,x):
        x = self.conv1(x)
        x= self.conv2(x)
        x= self.block1(x)
        x = self.block2(x)
        x = self.avgpool(x)
        x=x.view(-1,12544)
        x = self.classifer(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)   

In [7]:
class BasicConv2d_two(nn.Module):
    def __init__(self,in_,out_,f1,**kwargs):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_,out_, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
          )
            
            
    def forward(self,x):
        x = self.conv1(x)
        return x        

In [8]:
net3 = MyNet3()


In [24]:
summary(net3,input_size=(120,3,224,224),depth=3,device="cpu")

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [9]:
class EarlyStopping():
    def __init__(self, patience = 5, tol = 0.0005): #惯例地定义我们所需要的一切变量/属性\
        #当连续patience次迭代时，这一轮迭代的损失与历史最低损失之间的差值小于阈值时
        #就触发提前停止
        
        self.patience = patience
        self.tol = tol #tolerance，累积5次都低于tol才会触发停止
        self.counter = 0 #计数，计算现在已经累积了counter次
        self.lowest_loss = None
        self.early_stop = True #True - 提前停止，False - 不要提前停止
    
    def __call__(self,val_loss):
        if self.lowest_loss == None: #这是第一轮迭代
            self.lowest_loss = val_loss
        elif self.lowest_loss - val_loss > self.tol:
            self.lowest_loss = val_loss
            self.counter = 0
        elif self.lowest_loss - val_loss < self.tol:
            self.counter += 1
            print("\t NOTICE: Early stopping counter {} of {}".format(self.counter,self.patience))
            if self.counter >= self.patience:
                print('\t NOTICE: Early Stopping Actived')
                self.early_stop = True
        return self.early_stop
        #这一轮迭代的损失与历史最低损失之间的差 - 阈值

In [10]:
def IterOnce(net,criterion,opt,x,y):
    """
    对模型进行一次迭代的函数
    
    net: 实例化后的架构
    criterion: 损失函数
    opt: 优化算法
    x: 这一个batch中所有的样本
    y: 这一个batch中所有样本的真实标签
    """
    sigma = net.forward(x)
    loss = criterion(sigma,y)
    loss.backward()
    opt.step()
    opt.zero_grad(set_to_none=True) #比起设置梯度为0，让梯度为None会更节约内存
    yhat = torch.max(sigma,1)[1]
    correct = torch.sum(yhat == y)
    return correct,loss

In [11]:
def TestOnce(net,criterion,x,y):
    """
    对一组数据进行测试并输出测试结果的函数
    
    net: 经过训练后的架构
    criterion：损失函数
    x：要测试的数据的所有样本
    y：要测试的数据的真实标签
    """
    #对测试，一定要阻止计算图追踪
    #这样可以节省很多内存，加速运算
    with torch.no_grad(): 
        sigma = net.forward(x)
        loss = criterion(sigma,y)
        yhat = torch.max(sigma,1)[1]
        correct = torch.sum(yhat == y)
    return correct,loss

In [12]:
def fit_test(net,batchdata,criterion,opt,epochs,tol,modelname,PATH):
    """
    对模型进行训练，并在每个epoch后输出训练集和测试集上的准确率/损失
    以实现对模型的监控
    实现模型的保存
    
    参数说明：
    net: 实例化后的网络
    batchdata：使用Dataloader分割后的训练数据
    testdata：使用Dataloader分割后的测试数据
    criterion：所使用的损失函数
    opt：所使用的优化算法
    epochs：一共要使用完整数据集epochs次
    tol：提前停止时测试集上loss下降的阈值，连续5次loss下降不超过tol就会触发提前停止
    modelname：现在正在运行的模型名称，用于保存权重时作为文件名
    PATH：将权重文件保存在path目录下
    
    """
    
    SamplePerEpoch = batchdata.dataset.__len__() #整个epoch里有多少个样本
    allsamples = SamplePerEpoch*epochs
    trainedsamples = 0
    trainlosslist = []
    
#     early_stopping = EarlyStopping(tol=tol)
    highestacc = None
    
    for epoch in range(1,epochs+1):
        net.train()
        correct_train = 0
        loss_train = 0
        for batch_idx, (x, y) in enumerate(batchdata):
             #non_blocking 非阻塞=True，表示允许多个线程同时占用一个资源
            #一般来说，一段代码/数据占用一部分计算资源时，该资源是不对其他代码/数据开放的
            #此时被占用的资源叫做临界资源，正在运行的代码/数据被叫做临界区
            #设置non_blocking=True，相当于允许多个代码在临界资源上运行
            #可以加速运算
            x = x.to(device,non_blocking=True)
            y = y.to(device,non_blocking=True).view(x.shape[0])
           
            correct, loss = IterOnce(net,criterion,opt,x,y)
            trainedsamples += x.shape[0]
            loss_train += loss
            correct_train += correct
            
            if (batch_idx+1) % 125 == 0:
                #现在进行到了哪个epoch
                #现在训练到了多少个样本
                #总共要训练多少个样本
                #现在的训练的样本占总共需要训练的样本的百分比
                print('Epoch{}:[{}/{}({:.0f}%)]'.format(epoch
                                                       ,trainedsamples
                                                       ,allsamples
                                                       ,100*trainedsamples/allsamples))
            
        TrainAccThisEpoch = float(correct_train*100)/SamplePerEpoch
        TrainLossThisEpoch = float(loss_train)/SamplePerEpoch #平均每个样本上的损失
        trainlosslist.append(TrainLossThisEpoch)
    
       #清理GPU内存
        del x,y,correct,loss,correct_train,loss_train
        gc.collect()
        torch.cuda.empty_cache()
      
    
#         #每次训练完一个epoch，就在测试集上验证一下模型现在的效果
#         net.eval()
#         loss_test = 0
#         correct_test = 0
#         loss_test = 0
#         TestSample = testdata.dataset.__len__()

#         for x,y in testdata:
#             x = x.to(device,non_blocking=True)
#             y = y.to(device,non_blocking=True).view(x.shape[0])
           
#             correct, loss = TestOnce(net,criterion,x,y)
#             loss_test += loss
#             correct_test += correct

#         TestAccThisEpoch = float(correct_test * 100)/TestSample
#         TestLossThisEpoch = float(loss_test )/TestSample
#         testlosslist.append(TestLossThisEpoch)
        
#         del x,y,correct,loss,correct_test, loss_test
#         gc.collect()
#         torch.cuda.empty_cache()
        
        #对每一个epoch，打印训练和测试的结果
        #训练集上的损失，测试集上的损失，训练集上的准确率，测试集上的准确率
        print("\t Train Loss:{:.6f}, Train Acc:{:.3f}%".format(TrainLossThisEpoch
                                                            ,TrainAccThisEpoch
                                                                                 ))
        
        #如果测试集准确率出现新高/测试集loss出现新低，那我会保存现在的这一组权重
        if highestacc == None: #首次进行测试
            highestacc = TrainAccThisEpoch
        if highestacc < TrainAccThisEpoch:
            highestacc = TrainAccThisEpoch
            torch.save(net.state_dict(),os.path.join(PATH,modelname+".pt"))
#             torch.save(net.state_dict(), os.path.join(PATH, str(modelname) + ".pt"))
            print("\t Weight Saved")
        
#         #提前停止
#         early_stop = early_stopping(TestLossThisEpoch)
#         if early_stop == "True":
#             break
            
    print("Complete")
    return trainlosslist

In [13]:
def read_split_data(root: str, val_rate: float = 0.3):
    random.seed(420)  # 保证随机结果可复现
    assert os.path.exists(root), "dataset root: {} does not exist.".format(root)

    # 遍历文件夹，一个文件夹对应一个类别
    flower_class = [cla for cla in os.listdir(root) if os.path.isdir(os.path.join(root, cla))]
    # 排序，保证各平台顺序一致
    flower_class.sort()
    # 生成类别名称以及对应的数字索引
    class_indices = dict((k, v) for v, k in enumerate(flower_class))
    json_str = json.dumps(dict((val, key) for key, val in class_indices.items()), indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    train_images_path = []  # 存储训练集的所有图片路径
    train_images_label = []  # 存储训练集图片对应索引信息
    val_images_path = []  # 存储验证集的所有图片路径
    val_images_label = []  # 存储验证集图片对应索引信息
    every_class_num = []  # 存储每个类别的样本总数
    supported = [".jpg", ".JPG", ".png", ".PNG"]  # 支持的文件后缀类型
    # 遍历每个文件夹下的文件
    for cla in flower_class:
        cla_path = os.path.join(root, cla)
        # 遍历获取supported支持的所有文件路径
        images = [os.path.join(root, cla, i) for i in os.listdir(cla_path)
                  if os.path.splitext(i)[-1] in supported]
        # 排序，保证各平台顺序一致
        images.sort()
        # 获取该类别对应的索引
        image_class = class_indices[cla]
        # 记录该类别的样本数量
        every_class_num.append(len(images))
        # 按比例随机采样验证样本
#         val_path = random.sample(images, k=int(len(images) * val_rate))

        for img_path in images:
#             if img_path in val_path:  # 如果该路径在采样的验证集样本中则存入验证集
#                 val_images_path.append(img_path)
#                 val_images_label.append(image_class)
#             else:  # 否则存入训练集
            train_images_path.append(img_path)
            train_images_label.append(image_class)

    print("{} images were found in the dataset.".format(sum(every_class_num)))
    print("{} images for training.".format(len(train_images_path)))
   
    assert len(train_images_path) > 0, "number of training images must greater than 0."
  
    

    return train_images_path, train_images_label, val_images_path, val_images_label

In [14]:
class MyDataSet():
    """自定义数据集"""

    def __init__(self, images_path: list, images_class: list, transform=None):
        self.images_path = images_path
        self.images_class = images_class
        self.transform = transform

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, item):
        img = Image.open(self.images_path[item])
        # RGB为彩色图片，L为灰度图片
        if img.mode != 'RGB':
            raise ValueError("image: {} isn't RGB mode.".format(self.images_path[item]))
        label = self.images_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label
    
    
    
#     def __getitem__(self, item):
#         img = cv2.imread(self.images_path[item], cv2.IMREAD_GRAYSCALE)
#         if img is None:
#             raise ValueError("Failed to read image: {}".format(self.images_path[item]))
#         label = self.images_class[item]
    
#         if self.transform is not None:
#             img = self.transform(img)
    
#         return img, label
    
    
    
    

    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images, labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        return images, labels

In [15]:
data_path = "/home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_gro"

In [16]:
train_images_path, train_images_label, val_images_path, val_images_label = read_split_data(data_path)

3662 images were found in the dataset.
3662 images for training.


In [17]:
img_size = 224
data_transform = {
        "train": transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        "val": transforms.Compose([
#                                  transforms.Resize(int(img_size * 1.143)),
#                                    transforms.CenterCrop(img_size),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])}

# 实例化训练数据集
train_dataset = MyDataSet(images_path=train_images_path,
                              images_class=train_images_label,
                               transform=data_transform["train"]
                         )

# # 实例化验证数据集
# val_dataset = MyDataSet(images_path=val_images_path,
#                             images_class=val_images_label,
#                            transform=data_transform["val"]
#                        )



In [18]:
def full_procedure(net,epochs,bs,modelname, PATH, lr=0.001,alpha=0.99,gamma=0,wd=0,tol=10**(-5)):
    
    torch.manual_seed(1412)
    
    #分割数据
    batchdata = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=bs,
                                               shuffle=True,
                                               pin_memory=True,
                                               drop_last=True,
                                               collate_fn=train_dataset.collate_fn)

#     testdata =torch.utils.data.DataLoader(val_dataset,
#                                              batch_size=bs,
#                                              drop_last=True,
#                                              shuffle=False,
#                                              pin_memory=True,
                                             
#                                              collate_fn=val_dataset.collate_fn)
    
    #损失函数，优化算法
    criterion = nn.CrossEntropyLoss(reduction="sum") #进行损失函数计算时，最后输出结果的计算模式
    opt = optim.RMSprop(net.parameters(),lr=lr
                        ,alpha=alpha,momentum=gamma,weight_decay=wd)
    
    #训练与测试
    trainloss = fit_test(net,batchdata,criterion,opt,epochs,tol,modelname,PATH)
    
    return trainloss

In [19]:
#绘图函数
def plotloss(trainloss, testloss):
    plt.figure(figsize=(10, 7))
    plt.plot(trainloss, color="red", label="Trainloss")
    plt.plot(testloss, color="orange", label="Testloss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
#实例化
#trainloss, testloss = full_procedure(XXXX)
#plotloss()

In [27]:
modelname = "MyNet3_0.001_100_aptos_1"
torch.manual_seed(420)
model3 = MyNet3()
PATH  ="/home/ubuntu/fy/外部验证/aptos_data"
# model3.load_state_dict(torch.load("/home/ubuntu/fy/外部验证/aptos_data/MyNet3_0.05_aptos_2.pt"))
print(modelname)

net = model3.to(device,non_blocking=True)
trainloss3 = full_procedure(model3,100,128,modelname=modelname, PATH=PATH, lr=0.001,alpha=0.99,gamma=0.002,wd=0,tol=10**(-5))

MyNet3_0.001_50_aptos_1
	 Train Loss:0.921048, Train Acc:66.712%
	 Train Loss:0.323363, Train Acc:88.067%
	 Weight Saved
	 Train Loss:0.097591, Train Acc:95.767%
	 Weight Saved
	 Train Loss:0.056250, Train Acc:96.723%
	 Weight Saved
	 Train Loss:0.044551, Train Acc:96.723%
	 Train Loss:0.041277, Train Acc:96.778%
	 Weight Saved
	 Train Loss:0.040712, Train Acc:96.805%
	 Weight Saved
	 Train Loss:0.039826, Train Acc:96.887%
	 Weight Saved
	 Train Loss:0.034109, Train Acc:96.887%
	 Train Loss:0.039738, Train Acc:96.805%
	 Train Loss:0.037345, Train Acc:96.860%
	 Train Loss:0.039220, Train Acc:96.942%
	 Weight Saved
	 Train Loss:0.033414, Train Acc:96.832%
	 Train Loss:0.037641, Train Acc:96.696%
	 Train Loss:0.041726, Train Acc:96.860%
	 Train Loss:0.041613, Train Acc:96.668%
	 Train Loss:0.032909, Train Acc:96.914%
	 Train Loss:0.031980, Train Acc:96.969%
	 Weight Saved
	 Train Loss:0.037605, Train Acc:96.668%
	 Train Loss:0.033477, Train Acc:97.023%
	 Weight Saved
	 Train Loss:0.037024

In [24]:
torch.cuda.empty_cache()

In [17]:
gc.collect()
torch.cuda.empty_cache()